<a href="https://colab.research.google.com/github/James-Yuichi-Sato/swiss-bracket/blob/main/Swiss_Runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend

## Player Class

In [319]:
class Player():
  def __init__(self, name:str = "", id:int = 0):
    self.name = name
    self.id = id
    self.opponents = []
    self.wins = 0
    self.games = 0
    self.rounds = 0
    self.win_rate = None
    self.had_bye = False
  
  def get_win_rate(self):
    try:
      self.win_rate = float(self.wins/self.rounds)
    except Exception:
      self.win_rate = 0
    return self.win_rate

  def add_win(self, opp_id:int):
    self.wins += 1
    self.games += 1
    self.rounds += 1
    self.opponents.append(opp_id)
    return self.get_win_rate()
  
  def add_bye(self):
    self.wins += 1
    self.games += 1
    self.had_bye = True
    return self.get_win_rate()
  
  def add_loss(self, opp_id:int):
    self.rounds += 1
    self.games += 1
    self.opponents.append(opp_id)
    return self.get_win_rate()


## Swiss Bracket Class

In [353]:
import json, math, os
from collections import OrderedDict
from statistics import mean

class Swiss():
  """Swiss Bracket System
    Swiss Bracket System that assumes there are no ties.
    Follows the Monrad System
    https://en.wikipedia.org/wiki/Swiss-system_tournament#Monrad_system
    """
  def __init__(self, player_list = []):
    """Swiss Bracket Initialization
    Parameters:
    player_list (list(str)): List of Player Names to Put in the bracket

    Returns:
    None
    """
    self.player_database = OrderedDict()
    self.num_players = 0
    self.rounds = 0
    self.matchup_number = 1
    self.backup = {"players": {}, "brackets": {}, "drops": {}, "results": {}}
    self.dropped_players = []
    for player in player_list:
      self.add_player(player)
    self.forecast_rounds()

  def __len__(self):
    return self.num_players

  def _calculate_opponent_win_rate(self, player_ids):
    wrs = []
    for id in player_ids:
      wrs.append(self.player_database[id].get_win_rate())
    if not wrs:
      return 0
    return mean(wrs)

  def _generate_first_bracket(self):
    self.current_bracket = {"match_id": {"Player 1": "player_id", "Player 2": "player_id", "type": "match_or_bye"}}
    paired_ids = []
    ranked_ids = list(self.player_database.keys())
    for i in range(len(ranked_ids)):
      player_1_id = ranked_ids[i]
      not_bye = True
      if player_1_id not in self.dropped_players and player_1_id not in paired_ids:
        j = i+1
        try:
          player_2_id = ranked_ids[j]
        except:
          not_bye = False
        while player_2_id in self.dropped_players and not_bye:
          j += 1
          try:
            player_2_id = ranked_ids[j]
          except:
            not_bye = False
        if player_2_id in self.player_database.keys() and not_bye:
          matchup = {"Player 1": player_1_id, "Player 2": player_2_id, "type": "match"}
          paired_ids.append(player_1_id)
          paired_ids.append(player_2_id)
          self.current_bracket[self.matchup_number] = matchup
        else:
          matchup = {"Player 1": player_1_id, "Player 2": "None", "type": "bye"}
          paired_ids.append(player_1_id)
          self.current_bracket[self.matchup_number] = matchup
        self.matchup_number += 1
    try:
      self.current_bracket.pop("match_id")
    except:
      print("Key Not In Dictionary")

  def _verify_results(self, results: dict):
    if sorted(self.current_bracket.keys()) != sorted(results.keys()):
      return False
    for match_num, pairing in self.current_bracket.items():
      if results[match_num] not in pairing.values():
        return False
    return True

  def generate_bracket(self):
    """Generates the next bracket and outputs to console
    Parameters:
    None

    Returns:
    None
    """
    if self.rounds == 0:
      self._generate_first_bracket()
    else:
      self.list_rankings();
      self._generate_first_bracket()
    self.backup["brackets"][self.rounds+1] = self.current_bracket
    self.announce_bracket()

  def announce_bracket(self):
    """Outputs the current bracket to console
    Parameters:
    None

    Returns:
    None
    """
    print(f"----------------------------------")
    print(f"Brackets for Round {self.rounds+1}")
    print(f"----------------------------------")
    for match_number, pairing in self.current_bracket.items():
      print(f"Match Number: {match_number}")
      for name, player_id in pairing.items():
        if name == "type":
          continue
        if player_id == "None":
          print("---BYE---")
          print(f"------------------")
          continue
        player = self.player_database[player_id]
        print(f"---{name}---")
        print(f"Name: {player.name}")
        print(f"ID: {player.id}")
        print(f"Games Won: {player.wins}")
        op_wr = self._calculate_opponent_win_rate(player.opponents)
        print(f"Opponents' WR: {op_wr}")
      print(f"------------------")

  def print_rankings(self):
    """Orders the participant list by ranking and output to console
    Parameters:
    None

    Returns:
    None
    """
    self.list_rankings()
    place_num = 1
    print(f"------------------")
    for id, player_class in self.player_database.items():
      if id in self.dropped_players:
        continue
      print(f"{place_num} Ranked")
      print(f"Player Name: {player_class.name}")
      print(f"Player ID: {player_class.id}")
      print(f"Games Won: {player_class.wins}")
      print(f"Rounds Played: {player_class.rounds}")
      opp_wr = self._calculate_opponent_win_rate(player_class.opponents)
      print(f"Opponents' WR: {opp_wr}")
      print(f"------------------")
      place_num += 1

  def report_results(self, results: dict):
    """Report results for the most recently generated bracket

    Parameters:
    results (dict): Results of matchups in Match ID: Player ID pairs
                    Bye matches MUST be designated to the Bye player

    Returns:
    None
    """
    if not self._verify_results(results):
      print("Invalid Results, Check and Retry")
      return False
    print("Valid Results, Saving Results")
    for match_id, winner_id in results.items():
      if self.current_bracket[match_id]["type"] == "bye":
        self.player_database[winner_id].add_bye()
        continue
      player_1_id = self.current_bracket[match_id]["Player 1"]
      player_2_id = self.current_bracket[match_id]["Player 2"]
      if winner_id == player_1_id:
        self.player_database[player_1_id].add_win(player_2_id)
        self.player_database[player_2_id].add_loss(player_1_id)
      else:
        self.player_database[player_2_id].add_win(player_1_id)
        self.player_database[player_1_id].add_loss(player_2_id)
    self.backup["results"][self.rounds]=results
    print("Results saved, ready to generate next round or add/drop players")
    self.rounds += 1
    return True

  def save_backup(self, filename):
    filepath = os.path.join(os.getcwd(), filename)
    print(filepath)
    with open(filepath, "w") as outfile:
      json.dump(self.backup, outfile, indent=4)

  def forecast_rounds(self):
    """Outputs the expected number of rounds for this bracket
    Parameters:
    None

    Returns:
    None
    """
    self.expected_rounds = math.ceil(math.log(self.num_players, 2))
    print(f"Expecting {self.expected_rounds} rounds today")

  def add_player(self, player_name):
    """Add a player from the Swiss Bracket
    Parameters:
    player_name (str): The name of the Player
                       Results can be verified with list_player_data()

    Returns:
    None
    """
    self.player_database[self.num_players] = Player(player_name, self.num_players)
    self.backup["players"][self.num_players] = player_name
    print(f"Player: {player_name} - ID: {self.num_players}")
    self.num_players += 1

  def drop_player(self, player_id):
    """Drops a player from the Swiss Bracket
    Parameters:
    player_id (int): The ID of the Player, can be obtained with list_player_data()

    Returns:
    None
    """
    self.dropped_players.append(player_id)
    self.backup["drops"][player_id] = {"Name": self.player_database[player_id].name, "Round": self.rounds}
    self.player_database[player_id].name = "DROPPED - " + self.player_database[player_id].name 
    print(f"{self.player_database[player_id].name} - ID: {player_id} - Removed from future brackets")
  
  def list_player_data(self):
    """Prints out the current list of players in any random order
    Parameters:
    None

    Returns:
    None
    """
    print(f"------------------")
    for id, player_class in self.player_database.items():
      print(f"Player Name: {player_class.name}")
      print(f"Player ID: {player_class.id}")
      print(f"Games Won: {player_class.wins}")
      print(f"Rounds Played: {player_class.rounds}")
      wr = player_class.get_win_rate()
      print(f"Win Rate: {wr}")
      print(f"------------------")
  
  def list_rankings(self):
    """Orders the participant list by ranking
    Parameters:
    None

    Returns:
    None
    """
    players_ranked = []
    for i in range(self.rounds, -1, -1):
      players_sublist = {player_class: self._calculate_opponent_win_rate(player_class.opponents) for player_class in self.player_database.values() if player_class.wins == i}
      players_sublist = sorted(players_sublist.items(), key=lambda x:x[1], reverse=True)
      players_sublist = [(player_object[0].id, player_object[0]) for player_object in players_sublist]
      players_ranked.extend(players_sublist)
    self.player_database = OrderedDict(players_ranked)

# Setup Bracket

In [354]:
players = ['a', 'b', 'c', 'd','e','f','g','h','i','j','k','l','m',]

In [355]:
bracket = Swiss(players)
bracket.drop_player(3)
bracket.add_player('n')

Player: a - ID: 0
Player: b - ID: 1
Player: c - ID: 2
Player: d - ID: 3
Player: e - ID: 4
Player: f - ID: 5
Player: g - ID: 6
Player: h - ID: 7
Player: i - ID: 8
Player: j - ID: 9
Player: k - ID: 10
Player: l - ID: 11
Player: m - ID: 12
Expecting 4 rounds today
DROPPED - d - ID: 3 - Removed from future brackets
Player: n - ID: 13


# First Bracket

## Generate Bracket

In [356]:
bracket.generate_bracket()

----------------------------------
Brackets for Round 1
----------------------------------
Match Number: 1
---Player 1---
Name: a
ID: 0
Games Won: 0
Opponents' WR: 0
---Player 2---
Name: b
ID: 1
Games Won: 0
Opponents' WR: 0
------------------
Match Number: 2
---Player 1---
Name: c
ID: 2
Games Won: 0
Opponents' WR: 0
---Player 2---
Name: e
ID: 4
Games Won: 0
Opponents' WR: 0
------------------
Match Number: 3
---Player 1---
Name: f
ID: 5
Games Won: 0
Opponents' WR: 0
---Player 2---
Name: g
ID: 6
Games Won: 0
Opponents' WR: 0
------------------
Match Number: 4
---Player 1---
Name: h
ID: 7
Games Won: 0
Opponents' WR: 0
---Player 2---
Name: i
ID: 8
Games Won: 0
Opponents' WR: 0
------------------
Match Number: 5
---Player 1---
Name: j
ID: 9
Games Won: 0
Opponents' WR: 0
---Player 2---
Name: k
ID: 10
Games Won: 0
Opponents' WR: 0
------------------
Match Number: 6
---Player 1---
Name: l
ID: 11
Games Won: 0
Opponents' WR: 0
---Player 2---
Name: m
ID: 12
Games Won: 0
Opponents' WR: 0
-------

## Report Bracket

In [357]:
results = {1: 0, 2: 2, 3:5, 4: 7, 5: 9, 6: 11, 7: 13}
bracket.report_results(results)

Valid Results, Saving Results
Results saved, ready to generate next round or add/drop players


True

## Check Results

In [358]:
bracket.print_rankings()

------------------
1 Ranked
Player Name: a
Player ID: 0
Games Won: 1
Rounds Played: 1
Opponents' WR: 0.0
------------------
2 Ranked
Player Name: c
Player ID: 2
Games Won: 1
Rounds Played: 1
Opponents' WR: 0.0
------------------
3 Ranked
Player Name: f
Player ID: 5
Games Won: 1
Rounds Played: 1
Opponents' WR: 0.0
------------------
4 Ranked
Player Name: h
Player ID: 7
Games Won: 1
Rounds Played: 1
Opponents' WR: 0.0
------------------
5 Ranked
Player Name: j
Player ID: 9
Games Won: 1
Rounds Played: 1
Opponents' WR: 0.0
------------------
6 Ranked
Player Name: l
Player ID: 11
Games Won: 1
Rounds Played: 1
Opponents' WR: 0.0
------------------
7 Ranked
Player Name: n
Player ID: 13
Games Won: 1
Rounds Played: 0
Opponents' WR: 0
------------------
8 Ranked
Player Name: b
Player ID: 1
Games Won: 0
Rounds Played: 1
Opponents' WR: 1.0
------------------
9 Ranked
Player Name: e
Player ID: 4
Games Won: 0
Rounds Played: 1
Opponents' WR: 1.0
------------------
10 Ranked
Player Name: g
Player ID: 

# Second Bracket

## Generate Bracket

In [359]:
bracket.generate_bracket()

----------------------------------
Brackets for Round 2
----------------------------------
Match Number: 8
---Player 1---
Name: a
ID: 0
Games Won: 1
Opponents' WR: 0.0
---Player 2---
Name: c
ID: 2
Games Won: 1
Opponents' WR: 0.0
------------------
Match Number: 9
---Player 1---
Name: f
ID: 5
Games Won: 1
Opponents' WR: 0.0
---Player 2---
Name: h
ID: 7
Games Won: 1
Opponents' WR: 0.0
------------------
Match Number: 10
---Player 1---
Name: j
ID: 9
Games Won: 1
Opponents' WR: 0.0
---Player 2---
Name: l
ID: 11
Games Won: 1
Opponents' WR: 0.0
------------------
Match Number: 11
---Player 1---
Name: n
ID: 13
Games Won: 1
Opponents' WR: 0
---Player 2---
Name: b
ID: 1
Games Won: 0
Opponents' WR: 1.0
------------------
Match Number: 12
---Player 1---
Name: e
ID: 4
Games Won: 0
Opponents' WR: 1.0
---Player 2---
Name: g
ID: 6
Games Won: 0
Opponents' WR: 1.0
------------------
Match Number: 13
---Player 1---
Name: i
ID: 8
Games Won: 0
Opponents' WR: 1.0
---Player 2---
Name: k
ID: 10
Games Won: 0


## Report Bracket

In [360]:
results = {8: 0, 9: 5, 10:9, 11: 13, 12: 4, 13: 8, 14: 12}
bracket.report_results(results)

Valid Results, Saving Results
Results saved, ready to generate next round or add/drop players


True

## Check Results

In [361]:
bracket.print_rankings()

------------------
1 Ranked
Player Name: a
Player ID: 0
Games Won: 2
Rounds Played: 2
Opponents' WR: 0.25
------------------
2 Ranked
Player Name: f
Player ID: 5
Games Won: 2
Rounds Played: 2
Opponents' WR: 0.25
------------------
3 Ranked
Player Name: j
Player ID: 9
Games Won: 2
Rounds Played: 2
Opponents' WR: 0.25
------------------
4 Ranked
Player Name: n
Player ID: 13
Games Won: 2
Rounds Played: 1
Opponents' WR: 0.0
------------------
5 Ranked
Player Name: l
Player ID: 11
Games Won: 1
Rounds Played: 2
Opponents' WR: 1.0
------------------
6 Ranked
Player Name: c
Player ID: 2
Games Won: 1
Rounds Played: 2
Opponents' WR: 0.75
------------------
7 Ranked
Player Name: h
Player ID: 7
Games Won: 1
Rounds Played: 2
Opponents' WR: 0.75
------------------
8 Ranked
Player Name: m
Player ID: 12
Games Won: 1
Rounds Played: 1
Opponents' WR: 0.5
------------------
9 Ranked
Player Name: e
Player ID: 4
Games Won: 1
Rounds Played: 2
Opponents' WR: 0.25
------------------
10 Ranked
Player Name: i
Pl

# Third Bracket

## Generate Bracket

In [362]:
bracket.generate_bracket()

----------------------------------
Brackets for Round 3
----------------------------------
Match Number: 15
---Player 1---
Name: a
ID: 0
Games Won: 2
Opponents' WR: 0.25
---Player 2---
Name: f
ID: 5
Games Won: 2
Opponents' WR: 0.25
------------------
Match Number: 16
---Player 1---
Name: j
ID: 9
Games Won: 2
Opponents' WR: 0.25
---Player 2---
Name: n
ID: 13
Games Won: 2
Opponents' WR: 0.0
------------------
Match Number: 17
---Player 1---
Name: l
ID: 11
Games Won: 1
Opponents' WR: 1.0
---Player 2---
Name: c
ID: 2
Games Won: 1
Opponents' WR: 0.75
------------------
Match Number: 18
---Player 1---
Name: h
ID: 7
Games Won: 1
Opponents' WR: 0.75
---Player 2---
Name: m
ID: 12
Games Won: 1
Opponents' WR: 0.5
------------------
Match Number: 19
---Player 1---
Name: e
ID: 4
Games Won: 1
Opponents' WR: 0.25
---Player 2---
Name: i
ID: 8
Games Won: 1
Opponents' WR: 0.25
------------------
Match Number: 20
---Player 1---
Name: b
ID: 1
Games Won: 0
Opponents' WR: 1.5
---Player 2---
Name: g
ID: 6
Ga

## Report Bracket

In [363]:
results = {15: 0, 16: 9, 17:11, 18: 7, 19: 4, 20: 1, 21: 10}
bracket.report_results(results)

Valid Results, Saving Results
Results saved, ready to generate next round or add/drop players


True

## Check Results

In [364]:
bracket.print_rankings()

------------------
1 Ranked
Player Name: j
Player ID: 9
Games Won: 3
Rounds Played: 3
Opponents' WR: 0.7222222222222222
------------------
2 Ranked
Player Name: a
Player ID: 0
Games Won: 3
Rounds Played: 3
Opponents' WR: 0.4444444444444444
------------------
3 Ranked
Player Name: n
Player ID: 13
Games Won: 2
Rounds Played: 2
Opponents' WR: 0.6666666666666666
------------------
4 Ranked
Player Name: l
Player ID: 11
Games Won: 2
Rounds Played: 3
Opponents' WR: 0.6111111111111112
------------------
5 Ranked
Player Name: f
Player ID: 5
Games Won: 2
Rounds Played: 3
Opponents' WR: 0.5555555555555556
------------------
6 Ranked
Player Name: h
Player ID: 7
Games Won: 2
Rounds Played: 3
Opponents' WR: 0.5
------------------
7 Ranked
Player Name: e
Player ID: 4
Games Won: 2
Rounds Played: 3
Opponents' WR: 0.2222222222222222
------------------
8 Ranked
Player Name: c
Player ID: 2
Games Won: 1
Rounds Played: 3
Opponents' WR: 0.7777777777777778
------------------
9 Ranked
Player Name: m
Player ID:

# Fourth Bracket

## Generate Bracket

In [365]:
bracket.generate_bracket()

----------------------------------
Brackets for Round 4
----------------------------------
Match Number: 22
---Player 1---
Name: j
ID: 9
Games Won: 3
Opponents' WR: 0.7222222222222222
---Player 2---
Name: a
ID: 0
Games Won: 3
Opponents' WR: 0.4444444444444444
------------------
Match Number: 23
---Player 1---
Name: n
ID: 13
Games Won: 2
Opponents' WR: 0.6666666666666666
---Player 2---
Name: l
ID: 11
Games Won: 2
Opponents' WR: 0.6111111111111112
------------------
Match Number: 24
---Player 1---
Name: f
ID: 5
Games Won: 2
Opponents' WR: 0.5555555555555556
---Player 2---
Name: h
ID: 7
Games Won: 2
Opponents' WR: 0.5
------------------
Match Number: 25
---Player 1---
Name: e
ID: 4
Games Won: 2
Opponents' WR: 0.2222222222222222
---Player 2---
Name: c
ID: 2
Games Won: 1
Opponents' WR: 0.7777777777777778
------------------
Match Number: 26
---Player 1---
Name: m
ID: 12
Games Won: 1
Opponents' WR: 0.6666666666666666
---Player 2---
Name: b
ID: 1
Games Won: 1
Opponents' WR: 0.6666666666666666


## Report Bracket

In [366]:
results = {22: 9, 23: 13, 24:5, 25: 4, 26: 12, 27: 10, 28: 6}
bracket.report_results(results)

Valid Results, Saving Results
Results saved, ready to generate next round or add/drop players


True

## Check Results

In [367]:
bracket.print_rankings()

------------------
1 Ranked
Player Name: j
Player ID: 9
Games Won: 4
Rounds Played: 4
Opponents' WR: 0.7291666666666666
------------------
2 Ranked
Player Name: n
Player ID: 13
Games Won: 3
Rounds Played: 3
Opponents' WR: 0.5833333333333334
------------------
3 Ranked
Player Name: a
Player ID: 0
Games Won: 3
Rounds Played: 4
Opponents' WR: 0.5625
------------------
4 Ranked
Player Name: f
Player ID: 5
Games Won: 3
Rounds Played: 4
Opponents' WR: 0.5208333333333334
------------------
5 Ranked
Player Name: e
Player ID: 4
Games Won: 3
Rounds Played: 4
Opponents' WR: 0.2708333333333333
------------------
6 Ranked
Player Name: l
Player ID: 11
Games Won: 2
Rounds Played: 4
Opponents' WR: 0.7291666666666666
------------------
7 Ranked
Player Name: h
Player ID: 7
Games Won: 2
Rounds Played: 4
Opponents' WR: 0.6041666666666666
------------------
8 Ranked
Player Name: k
Player ID: 10
Games Won: 2
Rounds Played: 3
Opponents' WR: 0.5
------------------
9 Ranked
Player Name: m
Player ID: 12
Games W

## Check Backup

In [368]:
print(bracket.backup)
bracket.save_backup("backup.json")

{'players': {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n'}, 'brackets': {1: {1: {'Player 1': 0, 'Player 2': 1, 'type': 'match'}, 2: {'Player 1': 2, 'Player 2': 4, 'type': 'match'}, 3: {'Player 1': 5, 'Player 2': 6, 'type': 'match'}, 4: {'Player 1': 7, 'Player 2': 8, 'type': 'match'}, 5: {'Player 1': 9, 'Player 2': 10, 'type': 'match'}, 6: {'Player 1': 11, 'Player 2': 12, 'type': 'match'}, 7: {'Player 1': 13, 'Player 2': 'None', 'type': 'bye'}}, 2: {8: {'Player 1': 0, 'Player 2': 2, 'type': 'match'}, 9: {'Player 1': 5, 'Player 2': 7, 'type': 'match'}, 10: {'Player 1': 9, 'Player 2': 11, 'type': 'match'}, 11: {'Player 1': 13, 'Player 2': 1, 'type': 'match'}, 12: {'Player 1': 4, 'Player 2': 6, 'type': 'match'}, 13: {'Player 1': 8, 'Player 2': 10, 'type': 'match'}, 14: {'Player 1': 12, 'Player 2': 'None', 'type': 'bye'}}, 3: {15: {'Player 1': 0, 'Player 2': 5, 'type': 'match'}, 16: {'Player 1': 9, 'Player 2': 13, 'type':

# Fifth Bracket

## Generate Bracket

In [ ]:
bracket.generate_bracket()

----------------------------------
Brackets for Round 2
----------------------------------
Match Number: 8
---Player 1---
Name: a
ID: 0
Games Won: 1
Opponents' WR: 0.0
---Player 2---
Name: c
ID: 2
Games Won: 1
Opponents' WR: 0.0
------------------
Match Number: 9
---Player 1---
Name: f
ID: 5
Games Won: 1
Opponents' WR: 0.0
---Player 2---
Name: h
ID: 7
Games Won: 1
Opponents' WR: 0.0
------------------
Match Number: 10
---Player 1---
Name: j
ID: 9
Games Won: 1
Opponents' WR: 0.0
---Player 2---
Name: l
ID: 11
Games Won: 1
Opponents' WR: 0.0
------------------
Match Number: 11
---Player 1---
Name: n
ID: 13
Games Won: 1
Opponents' WR: 0
---Player 2---
Name: b
ID: 1
Games Won: 0
Opponents' WR: 1.0
------------------
Match Number: 12
---Player 1---
Name: e
ID: 4
Games Won: 0
Opponents' WR: 1.0
---Player 2---
Name: g
ID: 6
Games Won: 0
Opponents' WR: 1.0
------------------
Match Number: 13
---Player 1---
Name: i
ID: 8
Games Won: 0
Opponents' WR: 1.0
---Player 2---
Name: k
ID: 10
Games Won: 0


## Report Bracket

In [ ]:
results = {8: 0, 9: 5, 10:9, 11: 13, 12: 4, 13: 8, 14: 12}
bracket.report_results(results)

Valid Results, Saving Results
Results saved, ready to generate next round or add/drop players


True

## Check Results

In [ ]:
bracket.print_rankings()

------------------
1 Ranked
Player Name: a
Player ID: 0
Games Won: 2
Rounds Played: 2
Win Rate: 1.0
Opponents' WR: 0.25
------------------
2 Ranked
Player Name: f
Player ID: 5
Games Won: 2
Rounds Played: 2
Win Rate: 1.0
Opponents' WR: 0.25
------------------
3 Ranked
Player Name: j
Player ID: 9
Games Won: 2
Rounds Played: 2
Win Rate: 1.0
Opponents' WR: 0.25
------------------
4 Ranked
Player Name: n
Player ID: 13
Games Won: 2
Rounds Played: 2
Win Rate: 1.0
Opponents' WR: 0.0
------------------
5 Ranked
Player Name: c
Player ID: 2
Games Won: 1
Rounds Played: 2
Win Rate: 0.5
Opponents' WR: 0.75
------------------
6 Ranked
Player Name: h
Player ID: 7
Games Won: 1
Rounds Played: 2
Win Rate: 0.5
Opponents' WR: 0.75
------------------
7 Ranked
Player Name: l
Player ID: 11
Games Won: 1
Rounds Played: 2
Win Rate: 0.5
Opponents' WR: 0.75
------------------
8 Ranked
Player Name: m
Player ID: 12
Games Won: 1
Rounds Played: 2
Win Rate: 0.5
Opponents' WR: 0.5
------------------
9 Ranked
Player Name